In [46]:
from braket.tracking import Tracker
from braket.aws import AwsDevice
from braket.circuits import Circuit, gates, noises, observables
from braket.devices import LocalSimulator
from braket.parametric import FreeParameter

import numpy as np
from scipy.stats import unitary_group
from braket.circuits.noise_model import (
    GateCriteria,
    NoiseModel,
    ObservableCriteria,
)
from braket.circuits import Circuit, Observable, Gate, Noise
from braket.circuits.noises import (
    BitFlip,
    Depolarizing,
    TwoQubitDepolarizing,
    Kraus,
)
from braket.devices import LocalSimulator
import numpy as np
import math
import matplotlib.pyplot as plt
from collections import deque
from collections import defaultdict

t = Tracker().start()

# build a simple circuit
circ = Circuit().h(0).cnot(0,1)

# define a noise channel
noise = noises.BitFlip(probability=0.1)

# add noise to every gate in the circuit
circ.apply_gate_noise(noise)

# select the local noise simulator
device = LocalSimulator('braket_dm')

# run the circuit on the local simulator
task = device.run(circ, shots = 1000)

# visualize the results
result = task.result()
measurement = result.measurement_counts
print('measurement results:', measurement)
    

measurement results: Counter({'11': 412, '00': 395, '10': 108, '01': 85})


In [1]:
# Example graph representation
# Nodes represent qubits, and edges represent crosstalk errors between qubits
# nodes = {0: {'error': 0.1}, 1: {'error': 0.2}, ...}  # Node ID and its error percentage
# edges = {(0, 1): 0.01, (0, 2): 0.02, ...}  # Tuple of node IDs and the error of the edge
  # List of communities, each containing one node
def calculate_eii_ai(communities, edges):
    """
    Calculate e_ii and a_i for each community.
    
    Parameters:
    - communities: List of lists, where each sublist represents a community and contains node IDs.
    - edges: Dictionary with node ID pairs as keys and error percentages as values.
    
    Returns:
    - e_ii_dict: Dictionary of e_ii values for each community.
    - a_i_dict: Dictionary of a_i values for each community.
    """
    total_weight = sum(edges.values())
    e_ii_dict = {}
    a_i_dict = {}
    
    for index, community in enumerate(communities):
        internal_weight = 0
        total_community_weight = 0
        
        # Calculate internal weights and total connected weights for each community
        for i in community:
            for j in community:
                if i != j and (i, j) in edges:
                    internal_weight += edges[(i, j)]
            for k, v in edges.items():
                if i in k:
                    total_community_weight += v
        
        e_ii = internal_weight / total_weight
        a_i = total_community_weight / total_weight
        
        e_ii_dict[index] = e_ii
        a_i_dict[index] = a_i
        
    return e_ii_dict, a_i_dict

def calculate_reward(community1, community2, communities, edges, nodes):
    """
    Calculate the reward for merging two communities with additional reliability factors.

    Parameters:
    - community1, community2: Indices of the two communities to be merged.
    - communities: Current list of all communities.
    - edges: Dictionary with node ID pairs as keys and error percentages as values.
    - nodes: Dictionary with node IDs as keys and their errors as values.

    Returns:
    - Reward value for merging the two communities, including reliability adjustments.
    """
    # First, calculate original Q values as before
    e_ii_dict, a_i_dict = calculate_eii_ai(communities, edges)
    Q_origin = sum(e_ii_dict.values()) - sum(a_i ** 2 for a_i in a_i_dict.values())
    
    # Merge communities for Q_merged calculation
    merged_community = communities[community1] + communities[community2]
    temp_communities = [c for i, c in enumerate(communities) if i not in [community1, community2]]
    temp_communities.append(merged_community)
    e_ii_dict_merged, a_i_dict_merged = calculate_eii_ai(temp_communities, edges)
    Q_merged = sum(e_ii_dict_merged.values()) - sum(a_i ** 2 for a_i in a_i_dict_merged.values())
    
    # Calculate V: Average reliability of readout operations on the qubits of the two communities
    node_errors = [nodes[node]['error'] for node in merged_community]
    V = 1 - sum(node_errors) / len(node_errors)  # Reliability is 1 - average error

    # Calculate X: Average conditional reliability of CNOTs within each community
    # that have crosstalk CNOT in the other community
    internal_edges = [edges[edge] for edge in edges if edge[0] in merged_community and edge[1] in merged_community]
    if internal_edges:
        X = 1 - sum(internal_edges) / len(internal_edges)  # Reliability is 1 - average error
    else:
        X = 1  # Default to 1 if no internal edges, indicating perfect reliability
    E = 0.95
    omega = 0.4
    # Update the reward function F to include X*V
    F = (Q_merged - Q_origin) + (X * V *E*omega)
    
    return F


def merge_communities(communities, edges):
    if len(communities) <= 1:
        return communities
    
    best_reward = None
    to_merge = (None, None)

    # Iterate over all pairs of communities to find the best merge based on the reward
    for i in range(len(communities)):
        for j in range(i + 1, len(communities)):
            # Calculate the reward for merging these two communities
            reward = calculate_reward(i, j, communities, edges, nodes)

            # If this pair has the best reward so far, remember it
            if best_reward is None or reward > best_reward:
                best_reward = reward
                to_merge = (i, j)

    # Perform the merge if a beneficial merge is found
    if to_merge[0] is not None:
        new_community = communities[to_merge[0]] + communities[to_merge[1]]
        # Create a new list of communities with the merged community replacing the original ones
        updated_communities = [communities[i] for i in range(len(communities)) if i not in to_merge]
        updated_communities.append(new_community)

        # Recursively call the function with the updated list of communities
        return merge_communities(updated_communities, edges)
    else:
        # If no beneficial merge is found, return the current communities
        return communities

import random

# Generate nodes with random error percentages
nodes = {i: {'error': random.uniform(0.01, 0.1)} for i in range(9)}

# Generate edges with random error rates between some pairs of nodes
edges = {}
for i in range(8):  # Example connections, not fully connected graph
    for j in range(i + 1, 9):
        if random.random() > 0.5:  # Randomly decide if an edge should exist
            edges[(i, j)] = random.uniform(0.001, 0.01)

# Example output
# print("Nodes:", nodes)
# print("Edges:", edges)
communities = [[node] for node in nodes] 
merge_communities(communities, edges)



[[2, 1, 8, 7, 0, 4, 3, 5, 6]]

In [35]:
model = standard_noise_model()
fiddy = get_two_qubit_fidelity(model, 9)

In [36]:
import random

class HierarchyTree:
    def __init__(self, num_nodes):
        # Each node is initially its own community
        self.nodes = {i: {'error': random.uniform(0.01, 0.1)} for i in range(num_nodes)}
        fiddy = get_two_qubit_fidelity(model, len(self.nodes)) 
        self.edges = {(i, j): fiddy[i][j] for i in range(num_nodes) for j in range(i+1, num_nodes)}
        self.communities = {i: [i] for i in range(num_nodes)}
        self.tree = {}
          
        

    def merge_communities(self):
        while len(self.communities) > 1:
            best_delta_q = float('-inf')
            best_pair = None

            # Calculate the best pair to merge
            for i in self.communities:
                for j in self.communities:
                    if i != j:
                        delta_q = self.calculate_delta_q(i, j)
                        if delta_q > best_delta_q:
                            best_delta_q = delta_q
                            best_pair = (i, j)

            if best_pair:
                self._merge(best_pair)

    def calculate_delta_q(self, comm1, comm2):
        # Simplified version: Difference in internal vs. external connections
        # Implement the actual FN algorithm's delta Q calculation here
        return random.uniform(-0.01, 0.01)  # Placeholder

    def _merge(self, pair):
        new_comm = self.communities[pair[0]] + self.communities[pair[1]]
        new_comm_id = max(self.communities.keys()) + 1
        self.tree[new_comm_id] = {'children': pair, 'community': new_comm}
        del self.communities[pair[0]]
        del self.communities[pair[1]]
        self.communities[new_comm_id] = new_comm

    def get_hierarchy_tree(self):
        return self.tree


num_nodes = 10
ht = HierarchyTree(num_nodes)
ht.merge_communities()
print(ht.get_hierarchy_tree())


{10: {'children': (9, 6), 'community': [9, 6]}, 11: {'children': (8, 1), 'community': [8, 1]}, 12: {'children': (5, 11), 'community': [5, 8, 1]}, 13: {'children': (4, 10), 'community': [4, 9, 6]}, 14: {'children': (7, 2), 'community': [7, 2]}, 15: {'children': (12, 0), 'community': [5, 8, 1, 0]}, 16: {'children': (3, 13), 'community': [3, 4, 9, 6]}, 17: {'children': (14, 15), 'community': [7, 2, 5, 8, 1, 0]}, 18: {'children': (17, 16), 'community': [7, 2, 5, 8, 1, 0, 3, 4, 9, 6]}}


In [11]:
def calculate_fidelity(noise_model, qubit_1, qubit_2):
    # Takes input of two qubits and calculates entangled gate fidelity
    # qubit_1 and qubit_2 should be integer
    
    # set up device: Local Simulator
    device = LocalSimulator()
    device = LocalSimulator(backend="braket_dm")
    
    c = Circuit().i(0).i(1).i(2).i(3).i(4).i(5).i(6).i(7).i(8).i(9).i(10).h(qubit_1).cnot(qubit_1, qubit_2)
    
    c = noise_model.apply(c)
    
    # run circuit (execute single TASK)
    result = device.run(c, shots=10000).result()
    
    # get measurement shots
    counts = result.measurement_counts
    
    # Extract top two count values from dictionary
    fidelity_counts = tuple([value for key, value in sorted(counts.items(), key=lambda x: x[1], reverse=True)[:2]])
    
    
    fidelity = (fidelity_counts[0]+fidelity_counts[1])/10000
    
    return fidelity

In [12]:
def standard_noise_model():
    rng = np.random.default_rng()
    m = NoiseModel()
    
    two_q_depo_mu = 1 - 0.9311
    two_q_depo_sigma = 0.005
    bf_mu = 1 - 0.99752
    bf_sigma = 0.0015
    one_q_depo_mu = 1 - 0.9981
    one_q_depo_sigma = 0.00017
    for qi in range(11):
        z_bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        z_bf_prob = 0.0 if z_bf_prob < 0.0 else z_bf_prob
        
        bf_prob = bf_mu + bf_sigma * rng.standard_normal()
        bf_prob = 0.0 if bf_prob < 0.0 else bf_prob
        
        one_q_depo_prob = one_q_depo_mu + one_q_depo_sigma * rng.standard_normal()
        one_q_depo_prob = 0.0 if one_q_depo_prob < 0.0 else one_q_depo_prob
        
        m.add_noise(BitFlip(z_bf_prob), ObservableCriteria(observables=Observable.Z, qubits=qi))
        #m.add_noise(BitFlip(bf_prob), ObservableCriteria(qubits=qi))
        
        m.add_noise(Depolarizing(one_q_depo_prob), GateCriteria(qubits=qi))
        for qj in range(11):
            if not qj == qi:
                two_q_depo_prob = two_q_depo_mu + two_q_depo_sigma * rng.standard_normal()
                two_q_depo_prob = 0.0 if two_q_depo_prob < 0.0 else two_q_depo_prob
                
                m.add_noise(TwoQubitDepolarizing(two_q_depo_prob), GateCriteria(gates=[Gate.CNot, Gate.Swap, Gate.CPhaseShift], qubits=[qi, qj]))
    return m

In [13]:
def get_two_qubit_fidelity(noise_model, num_qubits):
    
    N = num_qubits
    qubits = range(N)
    fidelity_matrix = np.zeros((N, N))

    for i in range(N):
        for j in range(i+1, N):
            fidelity = calculate_fidelity(noise_model, qubits[i], qubits[j])
            fidelity_matrix[i][j] = fidelity
            fidelity_matrix[j][i] = fidelity  # Symmetric

    return fidelity_matrix

In [6]:
print(len(nodes))


9


In [37]:
def greedyE_star(coupling_strength_matrix, hardware_graph):
    
    # Convert the coupling strength matrix to an edge list with weights
    edges_with_weights = []
    for i in range(len(coupling_strength_matrix)):
        for j in range(i + 1, len(coupling_strength_matrix)):
            weight = coupling_strength_matrix[i][j]
            if weight > 0:
                edges_with_weights.append(((i, j), weight))
    # Sort the edges by weights in descending order
    edges_with_weights.sort(key=lambda x: x[1], reverse=True)
    # Initialize the mapping of logical to physical qubits
    qubit_mapping = {}
    unmapped_qubits = set(range(len(coupling_strength_matrix)))
    # Function to find the hardware location with maximum reliability
    def find_max_reliability_location():
        return 0
    # Function to calculate the total reliability for a given qubit mapping
    def calculate_total_reliability(qubit, mapped_qubits):
        return 0
    # Map the first edge with the highest weight
    edge, _ = edges_with_weights[0]
    qubit_mapping[edge[0]] = find_max_reliability_location()
    unmapped_qubits.remove(edge[0])
    # Map the rest of the qubits based on edge weights and reliability
    for edge, weight in edges_with_weights:
        for qubit in edge:
            if qubit in unmapped_qubits:
                # Find the best hardware location for the unmapped qubit
                best_location = None
                best_reliability = -1
                for potential_location in hardware_graph:
                    # Temporarily map the qubit to calculate reliability
                    temp_mapping = qubit_mapping.copy()
                    temp_mapping[qubit] = potential_location
                    reliability = calculate_total_reliability(qubit, temp_mapping)
                    if reliability > best_reliability:
                        best_reliability = reliability
                        best_location = potential_location
                # Finalize the mapping for the qubit
                qubit_mapping[qubit] = best_location
                unmapped_qubits.remove(qubit)
    return qubit_mapping

In [ ]:


# CSM = [[0, 2, 2, 2], [2, 0, 2, 2], [2, 2, 0, 2], [2, 2, 2, 0]]

# IL = {1: [(3, 2), (4, 4)], 0: [(0, 6)], 2: [(4, 4), (8, 2)], 3: [(2, 6)]}
def sort_IL(input_map):

    # Extract items from the map and sort them based on the first item of the tuple
    sorted_items = sorted(input_map.items(), key=lambda x: x[1][0])

    # Extract and return the sorted keys (rows)
    sorted_keys = [item[0] for item in sorted_items]
    return sorted_keys

#P array sorted 
P = sort_IL(IL) #this works trust

def ProgQdegree(int):
    degree = 0
    x = P[int]
    for i in range(len(CSM)):
        if CSM[x][i] != 0:
            degree+=1
    return degree
print(ProgQdegree(1))
        
#iterate through all physical qbits from subgraph and calculate average CNOT error rates
#get cnot error rates from all adjacent nodes to q, average them to find the CNOT error rate of that qbit

#PhysErrorRates[] 
#do math here
#adjacentCount = 0
#nodeCount = 0
#for node in subgraph:
    #for adjacent in node.adjacent:
        #PhysErrorRates[nodeCount] += adjacent.CNOTerror 
        #adjacentCount+=1
    #PhysErrorRates[nodeCount] = #PhysErrorRates[nodeCount]/adjacentCount
    #adjacentCount = 0
    #nodeCount +=1
    
#list C, canidate qbits -- physical qubits whose degree is greater than or equal to the degree of the first pending program qubit in P;
C = []
for node in subgraph.node:
    if node.degree >= P[0].degree:
        C.append(node)
    
#If C is not empty, select the one with the lowest possible error rate in C
#need to find a way to associate the error rate with C
lowest = 100 #set high on purpose
tempLowest = 100 #same as above
if C:
    for node in C:
        if node.AvgError < lowest:
            lowest = node.AvgError
            selectedNode = node
            
else:
    for node in C:
        calculation = 1/((1-node.AvgError)*node.degree)
        if calculation < tempLowest:
            tempLowest = calculation
            selectedNode = node

In [3]:
def involved_qubits(IG):
    setty = set([])
    for i in IG:
        for j in list(i.target):
            setty.add(j)
    return setty

def cs_matrix(result_array,bell):
    full = [set([int(entry) for entry in sublist]) for sublist in [list(item.target) for row in result_array for item in row]]
    #full set of 2qubits in gate operation
    coupling_strength_matrix=[[0 for _ in range(bell.qubit_count)] for _ in range(bell.qubit_count)]
    for i in range(bell.qubit_count): #i is chosen as the index of the matrix
        for j in range(bell.qubit_count): #j is chosen as the index of the matrix
            #i or j is arbitray as it is symmetric
            for pair in full:
                if pair==set([i,j]):
                    coupling_strength_matrix[i][j] += 1
    return coupling_strength_matrix

def algo_1(bell):

    inst_list = bell.instructions

    temp=[]
    for key, value in bell.moments.items():
        temp.append([key.time,value])


    grouped_instructions = defaultdict(list)

    for entry, instruction in temp:
        grouped_instructions[entry].append(instruction)

    result_array = list(grouped_instructions.values())

    #mine just takes a gate set
    IG = [] #1
    IL = defaultdict(list)#[[] for _ in range (bell.qubit_count)]  #2
    #ignoring 3
    Gate_Set_Index = 0 #4
    for i in result_array: #5
        Gates_To_Remove = [] #6
        for j in IG: #7
            if len(j.target) == 2: #8 double check
                Gates_To_Remove.append(j) #9
            #10
        #11
        for j in i: #12
            for k in list(j.target): #13
                #14
                if k in involved_qubits(IG): #15
                    IL[int(k)][-1][1] += 1 #16
                else: #17
                    IL[int(k)].append([Gate_Set_Index,1]) #18
                #19
            #20
        #21
        for j in Gates_To_Remove: #22
            IG.remove(j) #23
        #24
        for j in i: #25
            IG.append(j) #26
        #27
        Gate_Set_Index += 1 #28
    #29
    #ignoring 30
    return result_array, {k: list(map(tuple, v)) for k, v in IL.items()}, cs_matrix(result_array,bell)

In [34]:
Bernstein_Vazirani = Circuit().h(0).h(1).h(2).h(3).x(4).h(4).cnot(0,4).cnot(3,4).h(0).h(1).h(2).h(3)

gate_states, IL, CS_matrix = algo_1(Bernstein_Vazirani)
print(gate_states)

print(IL)
print(CS_matrix)

[[Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(0)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(2)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(3)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': X('qubit_count': 1), 'target': QubitSet([Qubit(4)]), 'control': QubitSet([]), 'control_state': (), 'power': 1)], [Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(4)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1), 'target': QubitSet([Qubit(1)]), 'control': QubitSet([]), 'control_state': (), 'power': 1), Instruction('operator': H('qubit_count': 1),

In [38]:
print(greedyE_star(CS_matrix, [7, 2, 5, 8, 1, 0, 3, 4, 9, 6]))


{0: 0, 4: 7, 3: 7}


In [39]:
def process_braket_circuit(circuit):
    gate_set = defaultdict(int)
    involvement_lists = defaultdict(list)
    all_qubits = set()
    for instruction in circuit.instructions:
        if len(instruction.target) == 2:  # Check for 2-qubit gates
            q1, q2 = instruction.target
            gate_set[(q1, q2)] += 1
            all_qubits.update([q1, q2])
            for qubit in [q1, q2]:
                if involvement_lists[qubit] and involvement_lists[qubit][-1][0] == len(involvement_lists[qubit]):
                    involvement_lists[qubit][-1] = (len(involvement_lists[qubit]), involvement_lists[qubit][-1][1] + 1)
                else:
                    involvement_lists[qubit].append((len(involvement_lists[qubit]) + 1, 1))
    num_qubits = max(all_qubits) + 1
    coupling_strength_matrix = [[2 if i != j else 0 for j in range(num_qubits)] for i in range(num_qubits)]
    return dict(gate_set), {k: list(map(tuple, v)) for k, v in involvement_lists.items()}, coupling_strength_matrix
mod_gates, blank, not_used = process_braket_circuit(Bernstein_Vazirani)
print(mod_gates)

{(Qubit(0), Qubit(4)): 1, (Qubit(3), Qubit(4)): 1}


In [40]:
def map_qubits(gate_set, involvement_lists, coupling_strength_matrix):
    # Helper function to calculate the degree of a qubit
    def calculate_degree(qubit, gate_set):
        return sum([interactions for (q1, q2), interactions in gate_set.items() if qubit in [q1, q2]])
    # Initialize mapped qubits dictionary and pending program qubits list
    mapped_qubits = {}
    pending_program_qubits = sorted(involvement_lists.keys(), key=lambda q: involvement_lists[q][0][0])
    # Select a high-degree physical qubit for initial mapping
    physical_qubits_degrees = [sum(row) for row in coupling_strength_matrix]
    high_degree_physical_qubit = physical_qubits_degrees.index(max(physical_qubits_degrees))
    # Map the first pending program qubit to the high-degree physical qubit
    first_program_qubit = pending_program_qubits.pop(0)
    mapped_qubits[first_program_qubit] = high_degree_physical_qubit
    # Function to find the most suitable physical qubit for mapping
    def find_suitable_physical_qubit(program_qubit):
        best_physical_qubit = None
        best_score = float('inf')
        for pq, mapped_pq in mapped_qubits.items():
            for neighbor_pq in range(len(coupling_strength_matrix[mapped_pq])):
                if coupling_strength_matrix[mapped_pq][neighbor_pq] > 0 and neighbor_pq not in mapped_qubits.values():
                    score = calculate_degree(program_qubit, gate_set) - coupling_strength_matrix[mapped_pq][neighbor_pq]
                    if score < best_score:
                        best_score = score
                        best_physical_qubit = neighbor_pq
        return best_physical_qubit
    # Map remaining program qubits to physical qubits
    while pending_program_qubits:
        program_qubit = pending_program_qubits.pop(0)
        suitable_physical_qubit = find_suitable_physical_qubit(program_qubit)
        if suitable_physical_qubit is not None:
            mapped_qubits[program_qubit] = suitable_physical_qubit
    return mapped_qubits

In [42]:
print(map_qubits(mod_gates, IL, CS_matrix))

{0: 4, 1: 0, 2: 3}


In [49]:
def bernstein_vazirani():
    circuit = Circuit().i(0).i(1).i(2).i(3).i(4).i(5).i(6).i(7).i(8).i(9).i(10).h(1).h(4).h(3).h(2).x(0).h(0).cnot(1,0).cnot(2,0).h(1).h(4).h(3).h(2)
    return circuit

In [52]:
def run_circuit_simulation(circuit, noise_model):
    device = LocalSimulator()
    device = LocalSimulator(backend="braket_dm")
    circuit = noise_model.apply(circuit)
    #print(c)
    # run circuit (execute single TASK)
    result = device.run(circuit, shots=1000).result()
    # get measurement shots
    counts = result.measurement_counts
    return counts

In [54]:
bv_circuit = bernstein_vazirani()
bv_counts = run_circuit_simulation(bv_circuit, model)

In [55]:
# Sort the dictionary by values and extract the top 8 values
highest_eight = sorted(bv_counts.values(), reverse=True)[:8]
bv_fidelity = sum(highest_eight) / 1000
bv_fidelity

0.989

In [56]:
def bernstein_vazirani():
    circuit = Circuit().h(0).h(1).h(2).h(3).x(4).h(4).cnot(0,4).cnot(3,4).h(0).h(1).h(2).h(3)
    return circuit

In [57]:
bv_circuit_bad = bernstein_vazirani()
bv_counts_bad = run_circuit_simulation(bv_circuit, model)

In [58]:
# Sort the dictionary by values and extract the top 8 values
highest_eight = sorted(bv_counts_bad.values(), reverse=True)[:8]
bv_fidelity_bad = sum(highest_eight) / 1000
bv_fidelity_bad

0.984